In [81]:
from pathlib import Path
import pandas as pd
import xarray as xr
import yaml
import numpy as np
from itertools import zip_longest

def load_global():
    with open("configs/global.yaml", "r") as f:
        return yaml.safe_load(f)
    
cfg = load_global()


## Target data

In [71]:
data_file = cfg["data_raw_dir"] + "/cfr_NUTS2-DE.csv"
df = pd.read_csv(data_file)

In [ ]:
# Create a mask: True if value is not NaN
mask = df["DE11"].notna()

# Assign a group id: whenever a NaN occurs, the group number increases
groups = mask.ne(mask.shift()).cumsum()

# Filter only groups where values are numbers, then count their sizes
lengths = df[mask].groupby(groups).size()

# Calculate the average length
average_length = lengths.mean()

In [ ]:
print("Average sequence length:", average_length)

Average sequence length: 12.860486946318568


In [ ]:
df["DED5"].fillna(0).mean()

np.float64(0.10664560958363106)

In [190]:
# filter for midnight and day
df_day = df[df['Date'].str.endswith('12:00:00')].copy()
df_day['Date'] = df_day['Date'].str[:-9]

# filter for 1980 to 2024
df_day = df_day[(df_day['Date'] >= '1980-01-01') & (df_day['Date'] <= '2024-12-31')]

## Training data

In [ ]:
ds_era5 = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
ds_pangu = xr.open_zarr('gs://weatherbench2/datasets/pangu/2018-2022_0012_0p25.zarr')
ds_neuralgcm = xr.open_zarr('gs://weatherbench2/datasets/neuralgcm_deterministic/2020-512x256.zarr')
ds_graphcast = xr.open_zarr('gs://weatherbench2/datasets/graphcast/2020/date_range_2019-11-16_2021-02-01_12_hours_derived.zarr')
ds_era5_hour = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr')

In [141]:
# Check which variables are in all datasets
valid_variables1 = [v for v in ds_era5.data_vars if 
                   #v in ds_era5_hour.data_vars and
                   v in ds_pangu.data_vars and 
                   v in ds_neuralgcm.data_vars and 
                   v in ds_graphcast.data_vars]

valid_variables2 = [v for v in ds_pangu.data_vars if 
                   v in ds_era5.data_vars and 
                   #v in ds_era5_hour.data_vars and
                   #v in ds_neuralgcm.data_vars and 
                   v in ds_graphcast.data_vars]
print("          Variables present in")
print(f"{"All datasets":<{22}} {"All but neuralgcm":<{22}}")
print("-" * (22 + 22))
for a, b in zip_longest(valid_variables1, valid_variables2, fillvalue=""):
    print(f"{a:<22} {b}")

          Variables present in
All datasets           All but neuralgcm     
--------------------------------------------
geopotential           10m_u_component_of_wind
specific_humidity      10m_v_component_of_wind
temperature            10m_wind_speed
u_component_of_wind    2m_temperature
v_component_of_wind    geopotential
wind_speed             mean_sea_level_pressure
                       specific_humidity
                       temperature
                       u_component_of_wind
                       v_component_of_wind
                       wind_speed


In [142]:
[v for v in ds_era5_hour.data_vars]

['10m_u_component_of_wind',
 '10m_v_component_of_wind',
 '2m_dewpoint_temperature',
 '2m_temperature',
 'angle_of_sub_gridscale_orography',
 'anisotropy_of_sub_gridscale_orography',
 'boundary_layer_height',
 'geopotential',
 'geopotential_at_surface',
 'high_vegetation_cover',
 'lake_cover',
 'land_sea_mask',
 'leaf_area_index_high_vegetation',
 'leaf_area_index_low_vegetation',
 'low_vegetation_cover',
 'mean_sea_level_pressure',
 'mean_surface_latent_heat_flux',
 'mean_surface_net_long_wave_radiation_flux',
 'mean_surface_net_short_wave_radiation_flux',
 'mean_surface_sensible_heat_flux',
 'mean_top_downward_short_wave_radiation_flux',
 'mean_top_net_long_wave_radiation_flux',
 'mean_top_net_short_wave_radiation_flux',
 'mean_vertically_integrated_moisture_divergence',
 'potential_vorticity',
 'sea_ice_cover',
 'sea_surface_temperature',
 'slope_of_sub_gridscale_orography',
 'snow_depth',
 'soil_type',
 'specific_humidity',
 'standard_deviation_of_filtered_subgrid_orography',
 'stan

In [146]:
## Check dimensions and resolution of datasets

datasets = {
    "ERA5": ds_era5,
    "ERA5 1h": ds_era5_hour,
    "Pangu": ds_pangu,
    "NeuralGCM": ds_neuralgcm,
    "GraphCast": ds_graphcast,
}

# Collect info about dimensions per dataset
rows = []
for name, ds in datasets.items():
    dims = ds.sizes  # dict: {dim_name: size}
    # Try to find matching dims; if missing -> set to None
    time = dims.get("time") or dims.get("times") or None
    lat = dims.get("lat") or dims.get("latitude") or None
    lon = dims.get("lon") or dims.get("longitude") or None
    lev = dims.get("level") or dims.get("lev") or dims.get("plev") or None
    lead = dims.get("prediction_timedelta") or dims.get("lead") or None

    rows.append([name, time, lat, lon, lev, lead])

# Create DataFrame
df_dims = pd.DataFrame(rows, columns=["Dataset", "Time", "Lat", "Lon", "Lev", "Lead Time"])

# find time resolutions
for name, ds in datasets.items():
    time = ds.coords.get("time")
    if time is not None:
        deltas = np.diff(time.values)  # differences between consecutive times
        unique_deltas = np.unique(deltas)

        #from ns to hours
        unique_deltas = [int(delta / np.timedelta64(1, 'h')) for delta in unique_deltas]
        df_dims.loc[df_dims["Dataset"] == name, "Time Res/h"] = str(unique_deltas)

    lead = ds.coords.get("prediction_timedelta")
    if lead is not None:
        deltas = np.diff(lead.values)  # differences between consecutive lead times
        unique_deltas = np.unique(deltas)

        #from ns to hours
        unique_deltas = [int(delta / np.timedelta64(1, 'h')) for delta in unique_deltas]
        df_dims.loc[df_dims["Dataset"] == name, "Lead Time Res/h"] = str(unique_deltas)
 

df_dims


,Dataset,Time,Lat,Lon,Lev,Lead Time,Time Res/h,Lead Time Res/h
0,ERA5,93544,721,1440,13,NaN,[6],NaN
1,ERA5 1h,561264,721,1440,37,NaN,[1],NaN
2,Pangu,3652,721,1440,13,40.0,[12],[6]
3,NeuralGCM,797,256,512,37,31.0,[12],[12]
4,GraphCast,886,721,1440,37,40.0,[12],[6]


In [158]:
print(ds_era5["wind_speed"].sel(level=850).isel(time=0).sel(latitude=30, longitude=0).values)
u = ds_era5["u_component_of_wind"].sel(level=850).isel(time=0).sel(latitude=30, longitude=0).values
v = ds_era5["v_component_of_wind"].sel(level=850).isel(time=0).sel(latitude=30, longitude=0).values
print(np.sqrt(u**2 + v**2))

9.845405
9.845405


In [ ]:
# Cut out Germany from hourly ERA5 data
ds_era5_hour_de = ds_era5_hour.sel(latitude=slice(55, 47), longitude=slice(5, 15))